In [1]:
import numpy as np
import gmsh
factory = gmsh.model.occ
model = gmsh.model

In [2]:
surf_int_path = r'/home/simoneponcioni/Documents/01_PHD/03_Methods/Meshing/Meshing/99_testing_prototyping/testing_transfinite_2D/C0002233_int_CORT_int_11_point_insertion.npy'
surf_ext_path = r'/home/simoneponcioni/Documents/01_PHD/03_Methods/Meshing/Meshing/99_testing_prototyping/testing_transfinite_2D/C0002233_int_CORT_ext_11_point_insertion.npy'

surf_int = np.load(surf_int_path)
surf_ext = np.load(surf_ext_path)

In [4]:
gmsh.initialize()

def offset_tags(entities):
    """
    get all entities of the model and add offset to make unique tags
    """
    for tag in entities:
        newTag_s = (tag[0], tag[1] + 1000)
        gmsh.model.setTag(dim=tag[0], tag=tag[1], newTag=newTag_s[1])
    return None

def surfaces_gmsh(x1, y1, z1, x2, y2 ,z2):
        """
        https://gitlab.onelab.info/gmsh/gmsh/-/issues/456
        https://bbanerjee.github.io/ParSim/fem/meshing/gmsh/quadrlateral-meshing-with-gmsh/
        """
        gmsh.option.setNumber("General.Terminal", 1)

        points1 = []
        for i in range(0, len(x1)):
            point1 = factory.addPoint(x1[i], y1[i], z1)
            points1 = np.append(points1, point1)
        points1 = np.append(points1, points1[0])
        #B1 = factory.addBSpline(range(1, len(points1)), degree=3)
        #C1 = factory.addCurveLoop([B1])
        
        points2 = []
        for i in range(0, len(x2)):
            point2 = factory.addPoint(x2[i], y2[i], z2)
            points2 = np.append(points2, point2)
        points2 = np.append(points2, points2[0])
        #B2 = factory.addBSpline(range(len(points1) + 1, len(points1) + len(points2)), degree=3)
        #C2 = factory.addCurveLoop([B2])

def finalize():
    factory.synchronize()
    offset_tags(factory.getEntities())
    gmsh.write('/home/simoneponcioni/Documents/01_PHD/03_Methods/Meshing/Meshing/99_testing_prototyping/testing_transfinite_2D/surf_11_point_insertion.geo_unrolled')
    gmsh.finalize()

surfaces_gmsh(surf_ext[:, 0], surf_ext[:, 1], 0, surf_int[:, 0], surf_int[:, 1], 0)
finalize()

Info    : Writing '/home/simoneponcioni/Documents/01_PHD/03_Methods/Meshing/Meshing/99_testing_prototyping/testing_transfinite_2D/surf_11_point_insertion.geo_unrolled'...
Info    : Done writing '/home/simoneponcioni/Documents/01_PHD/03_Methods/Meshing/Meshing/99_testing_prototyping/testing_transfinite_2D/surf_11_point_insertion.geo_unrolled'
